# 第五章 处理输入：思维链推理

 - [一、环境配置](#一、环境配置)
 - [二、思维链 Prompt](#二、思维链-Prompt)
 - [三、内心独白（Inner monologue）](#三、内心独白（Inner-monologue）)


在本章中，我们将专注于处理输入，即通过一系列步骤生成有用的输出。

有时，模型在回答特定问题之前需要进行详细地推理。如果您参加过我们之前的课程，您将看到许多这样的例子。有时，模型可能会因为过于匆忙得出结论而在推理过程中出错。因此，我们可以重新构思查询，要求模型在给出最终答案之前提供<span style="color:red">一系列相关的推理步骤</span>，这样它就可以更长时间、更深入地思考问题。

通常，我们称这种要求模型逐步推理问题的策略为<span style="color:red">思维链推理（chain of thought reasoning）</span>。

## 一、环境配置


同上一章，我们首先需要配置使用 OpenAI API 的环境

In [1]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
client = OpenAI(api_key=openai_api_key)

def get_completion_from_messages(messages, 
                                 model ="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    封装一个支持更多参数的自定义访问 gpt-4o-mini的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为o1-mini(ChatGPT)
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = client.chat.completions.create(
        model= model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message.content

## 二、思维链 Prompt

我们在这里要求模型在得出结论之前一步一步推理答案。
<span style="color:#00ff00">
本质上还是做prompt工程
</span>

In [3]:
delimiter = "####"
system_message = f"""
请按照以下步骤回答客户的查询。客户的查询将以四个井号（#）分隔，即 {delimiter}。

步骤 1:{delimiter} 首先确定用户是否正在询问有关特定产品或产品的问题。产品类别不计入范围。

步骤 2:{delimiter} 如果用户询问特定产品，请确认产品是否在以下列表中。所有可用产品：

产品：TechPro 超极本
类别：计算机和笔记本电脑
品牌：TechPro
型号：TP-UB100
保修期：1 年
评分：4.5
特点：13.3 英寸显示屏，8GB RAM，256GB SSD，Intel Core i5 处理器
描述：一款适用于日常使用的时尚轻便的超极本。
价格：$799.99

产品：BlueWave 游戏笔记本电脑
类别：计算机和笔记本电脑
品牌：BlueWave
型号：BW-GL200
保修期：2 年
评分：4.7
特点：15.6 英寸显示屏，16GB RAM，512GB SSD，NVIDIA GeForce RTX 3060
描述：一款高性能的游戏笔记本电脑，提供沉浸式体验。
价格：$1199.99

产品：PowerLite 可转换笔记本电脑
类别：计算机和笔记本电脑
品牌：PowerLite
型号：PL-CV300
保修期：1年
评分：4.3
特点：14 英寸触摸屏，8GB RAM，256GB SSD，360 度铰链
描述：一款多功能可转换笔记本电脑，具有响应触摸屏。
价格：$699.99

产品：TechPro 台式电脑
类别：计算机和笔记本电脑
品牌：TechPro
型号：TP-DT500
保修期：1年
评分：4.4
特点：Intel Core i7 处理器，16GB RAM，1TB HDD，NVIDIA GeForce GTX 1660
描述：一款功能强大的台式电脑，适用于工作和娱乐。
价格：$999.99

产品：BlueWave Chromebook
类别：计算机和笔记本电脑
品牌：BlueWave
型号：BW-CB100
保修期：1 年
评分：4.1
特点：11.6 英寸显示屏，4GB RAM，32GB eMMC，Chrome OS
描述：一款紧凑而价格实惠的 Chromebook，适用于日常任务。
价格：$249.99

步骤 3:{delimiter} 如果消息中包含上述列表中的产品，请列出用户在消息中做出的任何假设，例如笔记本电脑 X 比笔记本电脑 Y 大，或者笔记本电脑 Z 有 2 年保修期。

步骤 4:{delimiter} 如果用户做出了任何假设，请根据产品信息确定假设是否正确。

步骤 5:{delimiter} 如果用户有任何错误的假设，请先礼貌地纠正客户的错误假设（如果适用）。只提及或引用可用产品列表中的产品，因为这是商店销售的唯一五款产品。以友好的口吻回答客户。

使用以下格式回答问题：
步骤 1:{delimiter} <步骤 1的推理>
步骤 2:{delimiter} <步骤 2 的推理>
步骤 3:{delimiter} <步骤 3 的推理>
步骤 4:{delimiter} <步骤 4 的推理>
回复客户:{delimiter} <回复客户的内容>

请确保在每个步骤之间使用 {delimiter} 进行分隔。
"""

In [4]:
user_message = f"""BlueWave Chromebook 比 TechPro 台式电脑贵多少？"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

步骤 1:#### 用户正在询问关于特定产品（BlueWave Chromebook 和 TechPro 台式电脑）之间的价格比较。

步骤 2:#### BlueWave Chromebook 的价格为 $249.99，TechPro 台式电脑的价格为 $999.99。

步骤 3:#### 用户假设 BlueWave Chromebook 比 TechPro 台式电脑贵。

步骤 4:#### 这个假设是错误的，因为实际上，BlueWave Chromebook 的价格远低于 TechPro 台式电脑。

回复客户:#### 您好！实际上，BlueWave Chromebook 的价格为 $249.99，而 TechPro 台式电脑的价格为 $999.99。因此，BlueWave Chromebook 并不比 TechPro 台式电脑贵，反而便宜了 $749.00。如果您有其他问题或需要更多信息，请随时告诉我！


In [5]:
user_message = f"""你们卖电视机么"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

步骤 1:#### 用户在询问是否销售电视机，这并不属于特定产品的范围，而是一个产品类别的问题。
步骤 2:#### 用户没有询问特定产品，因此无法确认是否在列表中。
步骤 3:#### 无法列出用户的假设，因为没有涉及特定产品。
步骤 4:#### 无法验证假设，因为没有假设被提出。
回复客户:#### 很抱歉，我们目前只销售计算机和笔记本电脑，并不提供电视机。如果您对我们的计算机产品感兴趣，请随时询问！


In [6]:
user_message = f"""你们卖哪些游戏笔记本？介绍一下"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

步骤 1:#### 用户正在询问有关特定产品（游戏笔记本电脑）的信息。

步骤 2:#### 在可用产品列表中，只有一款游戏笔记本电脑，即 BlueWave 游戏笔记本电脑。

步骤 3:#### 用户没有做出任何假设。

步骤 4:#### 无需纠正假设，因为用户没有做出任何假设。

回复客户:#### 我们目前销售的游戏笔记本电脑是 BlueWave 游戏笔记本电脑。它的特点包括 15.6 英寸显示屏，16GB RAM，512GB SSD，以及 NVIDIA GeForce RTX 3060 显卡。这款笔记本电脑提供高性能，适合沉浸式游戏体验。价格为 $1199.99，保修期为 2 年。如果您需要更多信息，请随时询问！


In [7]:
user_message = f"""1000$可以买得起一台台式电脑吗"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

步骤 1:#### 用户在询问是否可以用1000美元购买一台台式电脑。

步骤 2:#### 在可用产品列表中，确实有一款台式电脑：TechPro 台式电脑，价格为999.99美元。

步骤 3:#### 用户假设1000美元可以买得起一台台式电脑。

步骤 4:#### 这个假设是正确的，因为TechPro 台式电脑的价格为999.99美元，正好在1000美元的预算内。

回复客户:#### 是的，您可以用1000美元购买一台台式电脑。TechPro 台式电脑的价格为999.99美元，非常接近您的预算。如果您需要更多信息，请随时询问！


## 三、内心独白（Inner monologue）

对于某些应用程序，模型的推理过程可能不适合与用户共享。例如，在辅导类应用程序中，我们可能希望鼓励学生自行解决问题，但模型对学生解决方案的推理过程可能会泄露答案。

内心独白是一种可以用来缓解这种情况的策略，这是一种<span style="color:red">隐藏模型推理过程</span>的高级方法。

内心独白的思想是让模型以一种不会透露答案的方式生成部分输出，这样用户就无法看到完整的推理过程。目标是将这些部分隐藏在一个结构化的格式中，使得传递它们变得容易。然后，在向用户呈现输出之前，对输出进行一些转化，使得只有部分输出是可见的。

<span style="color:#00ff00">
下述代码中的try部分利用分隔符实现隐藏推理过程，只保留最后一个分隔符之后的模型输出
</span>

In [8]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

是的，您可以用1000美元购买一台台式电脑。TechPro 台式电脑的价格为999.99美元，非常接近您的预算。如果您需要更多信息，请随时询问！


在下一章中，我们将学习一种处理复杂任务的新策略，即将复杂任务分解为一系列更简单的子任务，而不是试图在一个 Prompt 中完成整个任务。

